In [1]:
import os 
from ultralytics import YOLO
from pathlib import Path
import cv2
import csv
from roboflow import Roboflow

# OCR Scan images for Temp, Date and Time. If readable append to the Pivot Table from Roboflow_Detection.ipynb. Currently not working.

In [38]:
from PIL import Image
import pytesseract
import os

# Path to the tesseract executable (only needed on Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Directory containing the images
image_directory = r'D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach'
output_directory = r'D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR'

# Make sure the output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Loop through all files in the image directory
for filename in os.listdir(image_directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
        image_path = os.path.join(image_directory, filename)
        try:
            # Load the image
            image = Image.open(image_path)
            
            # Use Tesseract to do OCR on the image
            text = pytesseract.image_to_string(image)
            
            # Define output text file path
            text_file_path = os.path.join(output_directory, f"{os.path.splitext(filename)[0]}.txt")
            
            # Save the extracted text to a file
            with open(text_file_path, 'w') as file:
                file.write(text)
            
            print(f"Processed {filename} and saved text to {text_file_path}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

print("Processing complete.")


Processed MFDC9249.JPG and saved text to D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9249.txt
Processed MFDC9250.JPG and saved text to D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9250.txt
Processed MFDC9251.JPG and saved text to D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9251.txt
Processed MFDC9252.JPG and saved text to D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9252.txt
Processed MFDC9253.JPG and saved text to D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9253.txt
Processed MFDC9254.JPG and saved text to D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9254.txt
Processed MFDC9255.JPG and saved text to D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9255.txt
Processed MFDC9256.JPG and saved text to D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9256.txt
Processed MFDC9257.JPG and saved text to D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9257.txt
Processed MFDC9258.JPG and saved text to D:\Yo

In [40]:
import os
import pandas as pd
import re

# Define paths
pivot_file = r'D:\Yolo_Code\prtm_yolo\Code\detections_pivot_sum_by_class.csv'
ocr_directory = r'D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR'  # Path to the folder containing OCR .txt files
output_csv = r'D:\Yolo_Code\prtm_yolo\Code\detections_with_ocr.csv'

# Load the detections pivot file
detections_df = pd.read_csv(pivot_file)

print(detections_df)

# Function to extract OCR data from .txt file
def extract_ocr_data(ocr_txt_file):
    with open(ocr_txt_file, 'r') as file:
        ocr_data = file.read()
    
    # Use regex to extract temperature, location, date, and time from the OCR data
    ocr_pattern = r'@ (?P<temperature>\d+)[°F] (?P<location>\w+) (?P<day>\d{1,2}) (?P<month>\w+) (?P<year>\d{4}) (?P<time>[\d:]+ \w+)'
    ocr_match = re.search(ocr_pattern, ocr_data)

    if ocr_match:
        temperature = ocr_match.group('temperature')
        location = ocr_match.group('location')
        day = ocr_match.group('day')
        month = ocr_match.group('month')
        year = ocr_match.group('year')
        time = ocr_match.group('time')
        return temperature, location, day, month, year, time
    else:
        return None  # If no valid OCR data is found

# Loop through each filename in the detections pivot file
for index, row in detections_df.iterrows():
    filename = row['filename']  # Get the image filename
    
    # Construct the corresponding OCR .txt filename (assuming they have the same base name)
    ocr_txt_file = os.path.join(ocr_directory, filename.replace('.JPG', '.txt'))
    
    # Check if the OCR file exists
    if os.path.exists(ocr_txt_file):
        ocr_data = extract_ocr_data(ocr_txt_file)  # Extract OCR data from the .txt file
        
        if ocr_data:
            temperature, location, day, month, year, time = ocr_data
            
            # Add the OCR data to the current row in the dataframe
            detections_df.loc[index, 'temperature'] = temperature
            detections_df.loc[index, 'location'] = location
            detections_df.loc[index, 'date'] = f"{day} {month} {year}"
            detections_df.loc[index, 'time'] = time
        else:
            print(f"No valid OCR data found in {ocr_txt_file}")
    else:
        print(f"OCR file not found for {filename}")

# Save the updated dataframe to a new CSV file
detections_df.to_csv(output_csv, index=False)

print(f"Updated detections saved to: {output_csv}")


No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9249.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9250.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9251.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9252.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9253.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9254.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9255.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9256.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9257.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9258.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\M

In [41]:
import os
import pandas as pd
import re

# Define paths
pivot_file = r'D:\Yolo_Code\prtm_yolo\Code\detections_pivot_sum_by_class.csv'
ocr_directory = r'D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR'  # Path to the folder containing OCR .txt files
output_csv = r'D:\Yolo_Code\prtm_yolo\Code\detections_with_ocr.csv'

# Load the detections pivot file
detections_df = pd.read_csv(pivot_file)

print(detections_df)

        filename  Boat  Human  People
0   MFDC9249.JPG   0.0    2.0     0.0
1   MFDC9250.JPG   0.0   11.0     0.0
2   MFDC9251.JPG   1.0    2.0     0.0
3   MFDC9252.JPG   1.0    9.0     1.0
4   MFDC9253.JPG   2.0   17.0     0.0
..           ...   ...    ...     ...
94  MFDC9344.JPG   1.0   12.0     9.0
95  MFDC9345.JPG   1.0   23.0     7.0
96  MFDC9346.JPG   1.0   12.0     4.0
97  MFDC9347.JPG   1.0   23.0     6.0
98  MFDC9348.JPG   3.0   10.0     3.0

[99 rows x 4 columns]


In [49]:
# Function to extract OCR data from .txt file
def extract_ocr_data(ocr_txt_file):
    with open(ocr_txt_file, 'r') as file:
        ocr_data = file.read()
    
    # Debug: Print the OCR data for checking the format
    print(f"OCR data for {ocr_txt_file}:\n{ocr_data}\n")

    # Adjust regex based on the sample OCR data
    ocr_pattern = r'@ (?P<temperature>\d+)[°F] (?P<location>\w+)-\d (?P<day>\d{1,2}) (?P<month>\w+) (?P<year>\d{4}) (?P<time>[\d:]+ \w+)'
    ocr_match = re.search(ocr_pattern, ocr_data)

    if ocr_match:
        temperature = ocr_match.group('temperature')
        location = ocr_match.group('location')
        day = ocr_match.group('day')
        month = ocr_match.group('month')
        year = ocr_match.group('year')
        time = ocr_match.group('time')
        return temperature, location, day, month, year, time
    else:
        print(f"No match found in {ocr_txt_file}")
        return None  # If no valid OCR data is found

In [55]:
# Loop through each filename in the detections pivot file
for index, row in detections_df.iterrows():
    filename = row['filename']  # Get the image filename
    
    # Construct the corresponding OCR .txt filename (assuming they have the same base name)
    ocr_txt_file = os.path.join(ocr_directory, filename.replace('.JPG', '.txt'))
    
    # Check if the OCR file exists
    if os.path.exists(ocr_txt_file):
        ocr_data = extract_ocr_data(ocr_txt_file)  # Extract OCR data from the .txt file
        
        if ocr_data:
            temperature, location, day, month, year, time = ocr_data
            
            # Add the OCR data to the current row in the dataframe
            detections_df.loc[index, 'temperature'] = temperature
            detections_df.loc[index, 'location'] = location
            detections_df.loc[index, 'date'] = f"{day} {month} {year}"
            detections_df.loc[index, 'time'] = time

            print(ocr_data, "test")
            
        else:
            print(f"No valid OCR data found in {ocr_txt_file}")
    else:
        print(f"OCR file not found for {filename}")

OCR data for D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9249.txt:
a

@ 69°F LKANN-1 25 MAY 2024 07:30 am



No match found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9249.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9249.txt
OCR data for D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9250.txt:
@ 75°F LKANN-1 25 MAY 2024 08:00 am

nn i i



No match found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9250.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9250.txt
OCR data for D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9251.txt:
| - .
' r oe ae

25 MAY 2024 08:30 am



No match found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9251.txt
No valid OCR data found in D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9251.txt
OCR data for D:\Yolo_Code\prtm_yolo\datasets\Lake_Anna_Beach\OCR\MFDC9252.txt:
LKANN-1 2024 09:00 am



No m

In [53]:
# Save the updated dataframe to a new CSV file
detections_df.to_csv(output_csv, index=False)

print(f"Updated detections saved to: {output_csv}")

Updated detections saved to: D:\Yolo_Code\prtm_yolo\Code\detections_with_ocr.csv
